In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data from the CSV file
data = pd.read_csv("/content/drive/MyDrive/new/data.csv")

# Split the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Save the train and test sets as separate CSV files
train_data.to_csv("train.csv", index=False)
test_data.to_csv("test.csv", index=False)

In [ ]:
!pip install transformers


In [ ]:
import pandas as pd

# Load data from CSV file
data = pd.read_csv('/content/drive/MyDrive/new/data.csv')

# Extract text and labels from data
data.head()


In [ ]:
# Extract text and labels from data
text = data['tweet']
labels = data['label']

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

class LanguageModel(nn.Module):
    def __init__(self, num_classes):
        super(LanguageModel, self).__init__()
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        output = self.dropout(pooled_output)
        output = self.fc1(output)
        return output

class CapsuleNetwork(nn.Module):
    def __init__(self, num_classes):
        super(CapsuleNetwork, self).__init__()
        self.language_model = LanguageModel(num_classes)
        self.capsule_layer = nn.Sequential(
            nn.Linear(num_classes, 32),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(32, 16),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(16, num_classes * 16),
            nn.ReLU(inplace=True)
        )
        self.num_classes = num_classes
        self.capsule_activation = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
        output = self.language_model(input_ids, attention_mask)
        output = self.capsule_layer(output)
        output = output.view(-1, self.num_classes, 16)
        output = self.capsule_activation(output)
        class_capsules = torch.sum(output, dim=2)
        class_probabilities = F.softmax(class_capsules, dim=1)
        return class_probabilities


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize text
tokens = tokenizer.batch_encode_plus(
    text.tolist(),
    max_length=128,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# Extract input IDs and attention masks from tokens
input_ids = torch.tensor(tokens['input_ids'])
attention_masks = torch.tensor(tokens['attention_mask'])
labels = torch.tensor(labels.tolist())


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class CapsuleNetwork(nn.Module):
    def __init__(self, num_classes):
        super(CapsuleNetwork, self).__init__()
        self.num_classes = num_classes
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.capsule = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, num_classes)
        )

    def forward(self, input_ids, attention_masks):
        outputs = self.bert(input_ids, attention_mask=attention_masks)
        sequence_output = outputs.last_hidden_state
        pooled_output = outputs.pooler_output
        capsule_input = pooled_output.view(-1, 1, 768)
        capsule_output = self.capsule(capsule_input).squeeze()
        return capsule_output





In [ ]:
# Load data from CSV file
data = pd.read_csv('/content/drive/MyDrive/new/data.csv',nrows=500)
text = data['tweet']
labels = data['label']

# Tokenize text
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.batch_encode_plus(
    text.tolist(),
    max_length=128,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)
input_ids = torch.tensor(tokens['input_ids'])
attention_masks = torch.tensor(tokens['attention_mask'])
labels = torch.tensor(labels.tolist())

# Split data into training and testing sets
dataset = torch.utils.data.TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])


In [ ]:
input_ids

In [ ]:
attention_masks

In [ ]:
labels

In [ ]:
text

In [ ]:
# Instantiate the capsule network model
model = CapsuleNetwork(num_classes=2)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Train the model
batch_size = 32
num_epochs = 10
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train_losses = []
test_accuracies = []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        input_ids, attention_masks, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_masks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * input_ids.size(0)
    epoch_loss = running_loss / len(train_dataset)
    train_losses.append(epoch_loss)

    model.eval()
    num_correct = 0
    for batch in test_loader:
        input_ids, attention_masks, labels = batch
        with torch.no_grad():
            outputs = model(input_ids, attention_masks)
        preds = torch.argmax(outputs, dim=1)
        num_correct += torch.sum(preds == labels)
    accuracy = num_correct / len(test_dataset)
    test_accuracies.append(accuracy)

    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Plotting the graph
plt.plot(range(1, num_epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, num_epochs+1), test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Accuracy and Loss')
plt.legend()
plt.show()

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

training and performance evaluation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

# Define your model, criterion, optimizer, and data loaders
# ...

# Set up TensorBoard writer
writer = SummaryWriter()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_losses = []
test_accuracies = []

model = model.to(device)  # Move the model to the appropriate device
optimizer = optim.Adam(model.parameters())  # Initialize the optimizer with model parameters

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        input_ids, attention_masks, labels = batch
        optimizer.zero_grad()
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)
        outputs = model(input_ids, attention_masks)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * input_ids.size(0)
    epoch_loss = running_loss / len(train_dataset)
    train_losses.append(epoch_loss)

    model.eval()
    num_correct = 0
    predictions = torch.empty(0, dtype=torch.float32).to(device)
    targets = torch.empty(0, dtype=torch.float32).to(device)
    for batch in test_loader:
        input_ids, attention_masks, labels = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_masks)
        preds = torch.argmax(outputs, dim=1)
        num_correct += torch.sum(preds == labels)

        # Append predictions and targets for PR curve calculation
        predictions = torch.cat((predictions, outputs[:, 1].cpu().float().to(device)))
        targets = torch.cat((targets, labels.float().to(device)))

    accuracy = num_correct / len(test_dataset)
    test_accuracies.append(accuracy)

    print(f'Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Test Accuracy: {accuracy:.4f}')

    # Write accuracy and loss to TensorBoard as scalars
    writer.add_scalar('Metrics/Train Loss', epoch_loss, epoch)
    writer.add_scalar('Metrics/Test Accuracy', accuracy, epoch)

    # Write distribution of model parameters to TensorBoard
    for name, param in model.named_parameters():
        writer.add_histogram(name, param, epoch)

    # Calculate PR curve and write to TensorBoard
    precision, recall, _ = precision_recall_curve(targets.cpu(), predictions.cpu())
    writer.add_pr_curve('Metrics/PR Curve', torch.tensor(targets.cpu()), torch.tensor(predictions.cpu()), global_step=epoch)

# Close TensorBoard writer
writer.close()


In [ ]:
from torch.utils.tensorboard import SummaryWriter

# Load the TensorBoard log directory
log_dir = "/content/runs"

# Create a SummaryWriter to visualize the loaded data
writer = SummaryWriter(log_dir=log_dir)

# Close the SummaryWriter
writer.close()


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir runs
%reload_ext tensorboard

In [ ]:
%reload_ext tensorboard

In [ ]:
# Define a file path to save the model
PATH = "/content/drive/MyDrive/new/capsule_net new.pth"

# Save the model's state dictionary to a file
torch.save(model.state_dict(), PATH)

Testing

In [ ]:
import torch
from transformers import BertTokenizer, RobertaForSequenceClassification

# Load the saved capsule-net model
model_path = '/content/drive/MyDrive/new/capsule_net new.pth'
model_state_dict = torch.load(model_path)
model = RobertaForSequenceClassification.from_pretrained('roberta-base', state_dict=model_state_dict)

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Get real-time user input
while True:
    user_input = input('Enter a comment (or type "exit" to stop): ')
    if user_input == 'exit':
        break

    # Tokenize the user input
    tokenized_input = tokenizer.tokenize(user_input)
    input_ids = tokenizer.convert_tokens_to_ids(tokenized_input)
    input_ids = torch.tensor([input_ids])
    attention_masks = torch.where(input_ids != 0, torch.tensor(1), torch.tensor(0))

    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_masks)
        logits = outputs[0]
        predictions = torch.argmax(logits, dim=1)

    # Print the prediction
    for prediction in predictions:
        if prediction == 0:
            print('The comment is not toxic.')
        else:
            print('The comment is toxic.')
